In [6]:
import sys
import json
import urllib3
import psycopg2
from pgcopy import CopyManager
from datetime import datetime, date
from pydantic import BaseModel, Field
from config import Settings

In [2]:
settings = Settings()

DB_CONNECT = f'postgres://{settings.db_user}:{settings.db_password}@{settings.db_host}:5432/{settings.db_name}'


http = urllib3.PoolManager()

headers = {
    "Accept": "application/json",
    "X-App-Token": settings.tx_app_token
}

params = {
    "$order": "obligation_end_date_yyyymmdd,tabc_permit_number",
    "$limit": 50000,
    "$offset": 0
}

url = "https://data.texas.gov/resource/naix-2893.json"
county_url = "https://data.texas.gov/api/geospatial/48ag-x9aa?method=export&format=GeoJSON"

In [3]:
r = http.request(
    "GET",
    url,
    headers = headers,
    fields = params
)

In [4]:
sys.getsizeof(r.data)/1000000

38.509384

In [14]:
class Receipt(BaseModel):
    taxpayer_number: str = Field(None, max_length = 15)
    taxpayer_name: str = None
    taxpayer_address: str = None
    taxpayer_city: str = None
    taxpayer_state: str = None
    taxpayer_zip: str = None
    taxpayer_county: str = None
    location_number: str = None
    location_name: str = None
    location_address: str = None
    location_city: str = None
    location_state: str = None
    location_zip: str = None
    location_county: str = None
    inside_city_limits: bool = Field(None, alias = 'inside_outside_city_limits_code_y_n')
    tabc_permit_number: str = None
    responsibility_begin: datetime = Field(None, alias = 'responsibility_begin_date_yyyymmdd')
    responsibility_end: datetime = Field(None, alias = 'responsibility_end_date_yyyymmdd')
    obligation_end: datetime = Field(None, alias = 'obligation_end_date_yyyymmdd')
    liquor_receipts: int = None
    wine_receipts: int = None
    beer_receipts: int = None
    cover_charge_receipts: int = None
    total_receipts: int = None

    class Config:
        schema_extra = {
            "db_table_name": "bev_receipt"
        } 

In [20]:
Receipt.schema(by_alias=False)

{'title': 'Receipt',
 'type': 'object',
 'properties': {'taxpayer_number': {'title': 'Taxpayer Number',
   'maxLength': 15,
   'type': 'string'},
  'taxpayer_name': {'title': 'Taxpayer Name', 'type': 'string'},
  'taxpayer_address': {'title': 'Taxpayer Address', 'type': 'string'},
  'taxpayer_city': {'title': 'Taxpayer City', 'type': 'string'},
  'taxpayer_state': {'title': 'Taxpayer State', 'type': 'string'},
  'taxpayer_zip': {'title': 'Taxpayer Zip', 'type': 'string'},
  'taxpayer_county': {'title': 'Taxpayer County', 'type': 'string'},
  'location_number': {'title': 'Location Number', 'type': 'string'},
  'location_name': {'title': 'Location Name', 'type': 'string'},
  'location_address': {'title': 'Location Address', 'type': 'string'},
  'location_city': {'title': 'Location City', 'type': 'string'},
  'location_state': {'title': 'Location State', 'type': 'string'},
  'location_zip': {'title': 'Location Zip', 'type': 'string'},
  'location_county': {'title': 'Location County', 'typ

In [9]:
records = [Receipt(**x) for x in json.loads(r.data.decode('utf-8'))]

In [71]:
sys.getsizeof(records)/1000000

4.290008

In [10]:
create_table = """ CREATE TABLE IF NOT EXISTS bev_receipt(
    bev_receipt_id SERIAL PRIMARY KEY,
    taxpayer_number VARCHAR(15),
    taxpayer_name VARCHAR(100),
    taxpayer_address VARCHAR(100),
    taxpayer_city VARCHAR(20),
    taxpayer_state VARCHAR(2),
    taxpayer_zip VARCHAR(5),
    taxpayer_county VARCHAR(3),
    location_number VARCHAR(15),
    location_name VARCHAR(100),
    location_address VARCHAR(100),
    location_city VARCHAR(20),
    location_state VARCHAR(2),
    location_zip VARCHAR(5),
    location_county VARCHAR(3),
    inside_outside_city_limits VARCHAR(1),
    tabc_permit_number VARCHAR(10),
    responsibility_begin_date TIMESTAMP,
    responsibility_end_date TIMESTAMP,
    obligation_end_date TIMESTAMP,
    liquor_receipts INTEGER,
    wine_receipts INTEGER,
    beer_receipts INTEGER,
    cover_charge_receipts INTEGER,
    total_receipts INTEGER
)  """
conn = psycopg2.connect(DB_CONNECT)
cursor = conn.cursor()
cursor.execute(create_table)
conn.commit()
cursor.close()

In [11]:
table_cols = ('taxpayer_number',
 'taxpayer_name',
 'taxpayer_address',
 'taxpayer_city',
 'taxpayer_state',
 'taxpayer_zip',
 'taxpayer_county',
 'location_number',
 'location_name',
 'location_address',
 'location_city',
 'location_state',
 'location_zip',
 'location_county',
 'inside_outside_city_limits',
 'tabc_permit_number',
 'responsibility_begin_date',
 'responsibility_end_date',
 'obligation_end_date',
 'liquor_receipts',
 'wine_receipts',
 'beer_receipts',
 'cover_charge_receipts',
 'total_receipts')

In [12]:
conn = psycopg2.connect(DB_CONNECT)
mgr = CopyManager(conn, 'bev_receipt', table_cols)
mgr.copy([tuple(record.dict().values()) for record in records])
conn.commit()

In [55]:
tuple(x.dict().values())

('17509743203',
 'FIREBIRD DOWNTOWN, INC.',
 '1845 WOODALL RODGERS FWY STE 1100',
 'DALLAS',
 'TX',
 '75201',
 '57',
 '1',
 'EL FENIX RESTAURANT',
 '1601 MCKINNEY AVE',
 'DALLAS',
 'TX',
 '75202',
 '57',
 'Y',
 'MB061420',
 datetime.datetime(1993, 12, 1, 0, 0),
 None,
 datetime.datetime(2007, 1, 31, 0, 0),
 31065,
 1261,
 11020,
 0,
 43346)